# Procesamiento

In [2]:
import pandas as pd
from preprocess import impute_and_clean

### Read data

In [3]:
df_employees= pd.read_csv("../../data/bronce/hired_employees.csv")
df_employees

,employee_id,name,datetime,department_id,job_id
0,1,Harold Vogt,2021-11-07T02:48:42Z,2.0,96.0
1,2,Ty Hofer,2021-05-30T05:43:46Z,8.0,NaN
2,3,Lyman Hadye,2021-09-01T23:27:38Z,5.0,52.0
3,4,Lotti Crowthe,2021-10-01T13:04:21Z,12.0,71.0
4,5,Gretna Lording,2021-10-10T22:22:17Z,6.0,80.0
...,...,...,...,...,...
1994,1995,Goran Yong,2021-08-22T21:57:53Z,2.0,83.0
1995,1996,Cirstoforo Youings,2021-04-01T17:48:42Z,3.0,23.0
1996,1997,Wilek Yurkevich,2021-05-04T12:19:50Z,2.0,132.0
1997,1998,Jerry Yven,2021-10-03T14:12:50Z,7.0,100.0


In [5]:
df_employees.columns


Index(['employee_id', 'name', 'datetime', 'department_id', 'job_id'], dtype='object')

### Fast EDA

- Verificación rápida para saber si antes y después nos encontramos con valores nulos.

#### Employees df

In [6]:
df_employees.isnull().sum()

employee_id       0
name             19
datetime         14
department_id    21
job_id           16
dtype: int64

In [7]:
fecha_dummy = pd.to_datetime('2000-01-01')
df_employees_1 = impute_and_clean(df_employees, nan_threshold=0.5)
df_hired_employees_c = df_employees_1.fillna(fecha_dummy)


In [8]:
df_hired_employees_c.isnull().sum()


employee_id      0
name             0
datetime         0
department_id    0
job_id           0
dtype: int64

In [9]:
df_hired_employees_c.dtypes

employee_id        int64
name              object
datetime          object
department_id    float64
job_id           float64
dtype: object

In [10]:
df_hired_employees_c['datetime'] = pd.to_datetime(df_hired_employees_c['datetime'], errors='coerce')
df_hired_employees_c['year'] = df_hired_employees_c['datetime'].dt.year
df_hired_employees_c['quarter'] = df_hired_employees_c['datetime'].dt.to_period('Q')

df_hired_employees_c

C:\Users\jefai\AppData\Local\Temp\ipykernel_9208\3807299651.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_hired_employees_c['quarter'] = df_hired_employees_c['datetime'].dt.to_period('Q')


,employee_id,name,datetime,department_id,job_id,year,quarter
0,1,Harold Vogt,2021-11-07 02:48:42+00:00,2.0,96.000000,2021.0,2021Q4
1,2,Ty Hofer,2021-05-30 05:43:46+00:00,8.0,75.431165,2021.0,2021Q2
2,3,Lyman Hadye,2021-09-01 23:27:38+00:00,5.0,52.000000,2021.0,2021Q3
3,4,Lotti Crowthe,2021-10-01 13:04:21+00:00,12.0,71.000000,2021.0,2021Q4
4,5,Gretna Lording,2021-10-10 22:22:17+00:00,6.0,80.000000,2021.0,2021Q4
...,...,...,...,...,...,...,...
1994,1995,Goran Yong,2021-08-22 21:57:53+00:00,2.0,83.000000,2021.0,2021Q3
1995,1996,Cirstoforo Youings,2021-04-01 17:48:42+00:00,3.0,23.000000,2021.0,2021Q2
1996,1997,Wilek Yurkevich,2021-05-04 12:19:50+00:00,2.0,132.000000,2021.0,2021Q2
1997,1998,Jerry Yven,2021-10-03 14:12:50+00:00,7.0,100.000000,2021.0,2021Q4


In [11]:
df_hired_employees_c['department_id'] = df_hired_employees_c['department_id'].fillna(-1).astype(int)
df_hired_employees_c['job_id'] = df_hired_employees_c['job_id'].fillna(-1).astype(int)

# Asegurarnos que la columna 'year' también esté en formato entero
df_hired_employees_c['year'] = df_hired_employees_c['datetime'].dt.year.fillna(-1).astype(int)
df_hired_employees_c

,employee_id,name,datetime,department_id,job_id,year,quarter
0,1,Harold Vogt,2021-11-07 02:48:42+00:00,2,96,2021,2021Q4
1,2,Ty Hofer,2021-05-30 05:43:46+00:00,8,75,2021,2021Q2
2,3,Lyman Hadye,2021-09-01 23:27:38+00:00,5,52,2021,2021Q3
3,4,Lotti Crowthe,2021-10-01 13:04:21+00:00,12,71,2021,2021Q4
4,5,Gretna Lording,2021-10-10 22:22:17+00:00,6,80,2021,2021Q4
...,...,...,...,...,...,...,...
1994,1995,Goran Yong,2021-08-22 21:57:53+00:00,2,83,2021,2021Q3
1995,1996,Cirstoforo Youings,2021-04-01 17:48:42+00:00,3,23,2021,2021Q2
1996,1997,Wilek Yurkevich,2021-05-04 12:19:50+00:00,2,132,2021,2021Q2
1997,1998,Jerry Yven,2021-10-03 14:12:50+00:00,7,100,2021,2021Q4


In [ ]:
df_hired_employees_c_hired_2021 = df_hired_employees_c[df_hired_employees_c['year'] == 2021] #Filter

In [ ]:
df_hired_employees_c_hired_2021.dtypes

### Persist en Silver

In [ ]:
df_hired_employees_c.to_csv("../../data/silver/hired_employees_cleaned.csv")

#### Department df

In [ ]:
df_departments = pd.read_csv("../../data/bronce/departments.csv", names=["department_id","department_name"])
df_departments

In [ ]:
df_departments["department_name"].duplicated().sum()
df_departments["department_id"].duplicated().sum()

In [ ]:
df_departments["department_name"].isnull().sum()
df_departments["department_id"].isnull().sum()

#### Job df

In [ ]:
df_job = pd.read_csv("../../data/bronce/jobs.csv", names=["job_id","job_name"])
df_job

In [ ]:
df_job["job_name"].duplicated().sum()

In [ ]:
df_job["job_id"].duplicated().sum()

In [ ]:
df_job["job_name"].isnull().sum()

## Preguntas de negocio

1. *" Number of employees hired for each job and department in 2021 divided by quarter. The table must be ordered alphabetically by department and job."*

- Filtrar por año
- Joinear con jobs y departments
- Agrupar por departamento, nombre y quarter.
- Ordenar alfabeticamente



In [ ]:
jobs_df = pd.read_csv("../../data/silver/cleaned_jobs.csv")
departments_df = pd.read_csv("../../data/silver/cleaned_departments.csv")

In [ ]:
# Conteo de empleados agrupados por trabajo y departamento en el año 2021, y agrupado también por quarter. Tabla ordenada alfabeticamente.
df_hired_employees_c_filtered = df_hired_employees_c[df_hired_employees_c["datetime"].dt.year == 2021]

# join con job
merged_df = pd.merge(df_hired_employees_c_filtered, jobs_df, on="job_id")

# join con departments
merged_df = pd.merge(merged_df, departments_df, on="department_id")

# Group by department, job, and quarter and count the number of employees hired
result = merged_df.groupby(['department_name', 'job_name', 'quarter']).size().reset_index(name='num_employees_hired')
result_df = result.sort_values(by=['department_name', 'job_name', 'quarter'])
result_df


2. *"List of ids, name and number of employees hired of each department that hired more employees than the mean of employees hired in 2021 for all the departments, ordered by the number of employees hired (descending)"*


- Filtrar por 2021
- Agrupar por department_id y contar
- Calcular la media de empleados conctratador
- Filtrar departamentos con más contrataciones que el promedio de contrataciones
- Merge con departments para traer cuales son esos departamentos
- Ordenar alfabéticamente


In [ ]:
# Reutilizo el df anterior
df_hired_employees_grouped =  df_hired_employees_c_filtered.groupby(by=["department_id"]).size().reset_index(name='num_employees_hired')
mean_hired = df_hired_employees_grouped["num_employees_hired"].mean() #scalar
departments_df_above_mean = df_hired_employees_grouped[df_hired_employees_grouped["num_employees_hired"]>mean_hired]
result = departments_df_above_mean.merge(departments_df, on="department_id")
result_df = result[['department_id', 'department_name', 'num_employees_hired']].sort_values(by='num_employees_hired', ascending=False)
result_df